In [1]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

""" 
Fonte dos dados:
https://www.portaltransparencia.gov.br/download-de-dados/orcamento-despesa/

Extraído direto por ano:
https://www.portaltransparencia.gov.br/download-de-dados/orcamento-despesa/{ano}
"""

dataset_path = 'datasets/raw/'
dataset_base_name = 'OrcamentoDespesa'
available_years = [
    '2014',
    '2015',
    '2016',
    '2017',
    '2018',
    '2019',
    '2020',
    '2021',
    # '2022',  # Disponivel mas incompleto
]
columns = ['EXERCÍCIO', 'CÓDIGO ÓRGÃO SUPERIOR', 'NOME ÓRGÃO SUPERIOR',
       'CÓDIGO ÓRGÃO SUBORDINADO', 'NOME ÓRGÃO SUBORDINADO',
       'CÓDIGO UNIDADE ORÇAMENTÁRIA', 'NOME UNIDADE ORÇAMENTÁRIA',
       'CÓDIGO FUNÇÃO', 'NOME FUNÇÃO', 'CÓDIGO SUBFUNÇÃO', 'NOME SUBFUNÇÃO',
       'CÓDIGO PROGRAMA ORÇAMENTÁRIO', 'NOME PROGRAMA ORÇAMENTÁRIO',
       'CÓDIGO AÇÃO', 'NOME AÇÃO', 'CÓDIGO CATEGORIA ECONÔMICA',
       'NOME CATEGORIA ECONÔMICA', 'CÓDIGO GRUPO DE DESPESA',
       'NOME GRUPO DE DESPESA', 'CÓDIGO ELEMENTO DE DESPESA',
       'NOME ELEMENTO DE DESPESA', 'ORÇAMENTO INICIAL (R$)',
       'ORÇAMENTO ATUALIZADO (R$)', 'ORÇAMENTO EMPENHADO (R$)',
       'ORÇAMENTO REALIZADO (R$)']

"""
Em 2020 houve uma despesa com CATEGORIA ECONÔMICA e GRUPO DE DESPESA inválidos.
Isto faz com que o pandas assuma que a coluna é do tipo `object`, gerando erros na junção dos datasets, 
portanto as colunas 'CÓDIGO CATEGORIA ECONÔMICA' e 'CÓDIGO GRUPO DE DESPESA' serão convertidas para `object`
"""
dtype_conversions = {
    'CÓDIGO CATEGORIA ECONÔMICA': 'object',
    'CÓDIGO GRUPO DE DESPESA': 'object',
}

raw_df = pd.DataFrame(columns=columns)
df_shapes = []

for year in available_years:
    ydf = pd.read_csv(f"{dataset_path}{year}_{dataset_base_name}.csv", sep=';', encoding='Latin-1')
    ydf = ydf.astype(dtype_conversions)

    raw_df = pd.merge(raw_df, ydf, how='outer')
    df_shapes.append(ydf.shape)

/tmp/ipykernel_8494/2233467554.py:56: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  raw_df = pd.merge(raw_df, ydf, how='outer')
/tmp/ipykernel_8494/2233467554.py:56: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  raw_df = pd.merge(raw_df, ydf, how='outer')
/tmp/ipykernel_8494/2233467554.py:56: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  raw_df = pd.merge(raw_df, ydf, how='outer')
/tmp/ipykernel_8494/2233467554.py:56: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  raw_df = pd.merge(raw_df, ydf, how='outer')
/tmp/ipykernel_8494/2233467554.py:56: FutureWarning: In a future

In [2]:
# Checagem simples da quantidade de linhas, para ter mais confiabilidade no dataset final
total_line_count = 0
for shape in df_shapes:
    total_line_count += shape[0]

print(total_line_count, raw_df.shape)
print(total_line_count == raw_df.shape[0])

# Mantém cópia do dataset original por segurança
df = raw_df

214908 (214908, 25)
True


In [3]:
# Remove colunas de CODIGO, pois são redundantes
code_columns = [col for col in df.columns if 'CÓDIGO' in col]
df = df.drop(code_columns, axis=1)

In [4]:
# Corrige separador decimal
df['ORÇAMENTO INICIAL (R$)'] = df['ORÇAMENTO INICIAL (R$)'].str.replace(',', '.')
df['ORÇAMENTO ATUALIZADO (R$)'] = df['ORÇAMENTO ATUALIZADO (R$)'].str.replace(',', '.')
df['ORÇAMENTO EMPENHADO (R$)'] = df['ORÇAMENTO EMPENHADO (R$)'].str.replace(',', '.')
df['ORÇAMENTO REALIZADO (R$)'] = df['ORÇAMENTO REALIZADO (R$)'].str.replace(',', '.')

# Converte colunas de ORÇAMENTO em float
budget_columns = [col for col in df.columns if 'ORÇAMENTO ' in col]
budget_dtypes_dict = { col:'float' for col in budget_columns }
df = df.astype(budget_dtypes_dict)

In [5]:
# Renomeia colunas
column_names = {
    'EXERCÍCIO': 'exercicio', 
    'NOME ÓRGÃO SUPERIOR': 'nome_orgao_superior', 
    'NOME ÓRGÃO SUBORDINADO': 'nome_orgao_subordinado', 
    'NOME UNIDADE ORÇAMENTÁRIA': 'nome_unidade_orcamentaria', 
    'NOME FUNÇÃO': 'nome_funcao', 
    'NOME SUBFUNÇÃO': 'nome_subfuncao', 
    'NOME PROGRAMA ORÇAMENTÁRIO': 'nome_programa_orcamentario', 
    'NOME AÇÃO': 'nome_acao', 
    'NOME CATEGORIA ECONÔMICA': 'nome_categoria_economica', 
    'NOME GRUPO DE DESPESA': 'nome_grupo_despesa', 
    'NOME ELEMENTO DE DESPESA': 'nome_elemento_despesa', 
    'ORÇAMENTO INICIAL (R$)': 'orcamento_inicial',
    'ORÇAMENTO ATUALIZADO (R$)': 'orcamento_atualizado',
    'ORÇAMENTO EMPENHADO (R$)': 'orcamento_empenhado',
    'ORÇAMENTO REALIZADO (R$)': 'orcamento_realizado',
}

df.rename(columns=column_names, inplace=True)

In [6]:
# Visualização de informações básicas do dataset
print(df.info())
df.sample(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214908 entries, 0 to 214907
Data columns (total 15 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   exercicio                   214908 non-null  int64  
 1   nome_orgao_superior         214908 non-null  object 
 2   nome_orgao_subordinado      214908 non-null  object 
 3   nome_unidade_orcamentaria   214908 non-null  object 
 4   nome_funcao                 214908 non-null  object 
 5   nome_subfuncao              214908 non-null  object 
 6   nome_programa_orcamentario  214908 non-null  object 
 7   nome_acao                   214908 non-null  object 
 8   nome_categoria_economica    214908 non-null  object 
 9   nome_grupo_despesa          214908 non-null  object 
 10  nome_elemento_despesa       214908 non-null  object 
 11  orcamento_inicial           214908 non-null  float64
 12  orcamento_atualizado        214908 non-null  float64
 13  orcamento_empe

,exercicio,nome_orgao_superior,nome_orgao_subordinado,nome_unidade_orcamentaria,nome_funcao,nome_subfuncao,nome_programa_orcamentario,nome_acao,nome_categoria_economica,nome_grupo_despesa,nome_elemento_despesa,orcamento_inicial,orcamento_atualizado,orcamento_empenhado,orcamento_realizado
87581,2017,Ministério da Economia,Instituto de Pesquisa Econômica Aplicada,INSTITUTO DE PESQUISA ECONOMICA APLICADA,Administração,Atenção básica,PROGRAMA DE GESTAO E MANUTENCAO DO MINISTERIO ...,ASSISTENCIA MEDICA E ODONTOLOGICA AOS SERVIDOR...,DESPESAS CORRENTES,Outras Despesas Correntes,Indenizações e Restituições,2046454.35,2046454.35,2046454.35,2026454.35
198772,2021,Ministério da Educação,Instituto Federal do Espírito Santo,"INST.FED.DE EDUC.,CIENC.E TEC.DO ESP.SANTO",Educação,Ensino profissional,EDUCACAO PROFISSIONAL E TECNOLOGICA,FUNCIONAMENTO DAS INSTITUICOES DA REDE FEDERAL...,DESPESAS CORRENTES,Outras Despesas Correntes,Auxílio Financeiro a Pesquisadores,219664.11,219664.11,219664.11,205964.11
206472,2021,Ministério das Relações Exteriores,Fundação Alexandre de Gusmão,FUNDACAO ALEXANDRE DE GUSMAO,Relações exteriores,Administração geral,PROGRAMA DE GESTAO E MANUTENCAO DO PODER EXECU...,ATIVOS CIVIS DA UNIAO,DESPESAS CORRENTES,Pessoal e Encargos Sociais,Contribuição a Entidades Fechadas de Previdência,5000.00,5000.00,3623.66,3335.86
73919,2016,Ministério da Saúde,Fundação Oswaldo Cruz,FUNDACAO OSWALDO CRUZ,Saúde,Difusão do conhecimento científico e tecnológico,FORTALECIMENTO DO SISTEMA UNICO DE SAUDE (SUS),COMUNICACAO E INFORMACOES PARA A EDUCACAO EM S...,DESPESAS CORRENTES,Outras Despesas Correntes,Passagens e Despesas com Locomoção,150524.93,150524.93,150524.93,138174.93
212589,2021,Ministério do Desenvolvimento Regional,Empresa de Trens Urbanos de Porto Alegre,EMPRESA DE TRENS URBANOS DE PORTO ALEGRE S/A,Urbanismo,Administração geral,PROGRAMA DE GESTAO E MANUTENCAO DO PODER EXECU...,ATIVOS CIVIS DA UNIAO,DESPESAS CORRENTES,Pessoal e Encargos Sociais,Obrigações Patronais,10205759.55,10205759.55,9350581.19,8814137.51


In [8]:
# Exporta o arquivo pré-processado, com dados agregados de todos os anos
from datetime import datetime

output_path = 'datasets/preprocessed/'
t = datetime.now()

df.to_csv(f"{output_path}{dataset_base_name}_{t}.csv", sep=';', encoding='utf-8', index = False)